# Potato Plant Diseases

Este dataset está compuesto por 2 carpetas en las que tenemos imágenes de plantas de patata, enfermas, sanas y con la enfermedad presente pero no avanzada, el objetivo de AIDE será conseguir clasificar las imágenes entre las enfermas, sanas y medio_enfermas.

Para que este repositorio no pese demasiado hay algunos fichero que he añadido al .gitignore para que quien se lo descargue no sufra mucho, además en todos digo de donde son por si quieren hacer pruebaas.

Este dataset es de kaggle <a href="https://www.kaggle.com/datasets/hafiznouman786/potato-plant-diseases-data/data">Potato Plant Diseases Data</a>

In [3]:
import aide

Definimos los parámetros del algoritmo

In [10]:
# Definimos las variables que vamos a utilizar
data_dir = "./Data/PlantVillage" # for mac
data_dir_win= "AIDE-POTATO_DISEASES\Data\PlantVillage"  # for windows
goal = """
The folder contains 3 inside folders named Potato__Early_blight, Potato__healthy and Potato__Late_blight, those are thee folders which inside are the images labeled as the folder which they are.

Make a classifier for the images using CNN
"""
eval="f1-score, auc-score, accuracy"

Arrancamos

In [11]:
exp = aide.Experiment(
    data_dir=data_dir,
    goal=goal
)

In [12]:
best_solution = exp.run(steps=10)

PermissionError: [WinError 5] Acceso denegado

La ejecución ha costado <label style="color:red">$0.03+ $0.09 = $0.12</label> y aunque después de 1h de ejecución del código, nos ha dado un error en los procesos pero no lo creo releveante ya que aún así nos ha dado 4 distintas opciones para la resolución del ejercicio así como el código que ha usado. Lo único malo es que no hemos conseguido la validación de la solución comparada al resto pero podemos obviarlo si este sale bien. Procedemos a probar el código.

Este tipo de errores suelen ocurrir cuando no especificamos bien lo que queremos hacer con el conjunto de datos o pedimos cosas que no podemos trabajar a la vez.

Sacamos tanto el código como la matriz de validación de la mejor opción

In [ ]:
print(f" La mejor solución tiene una validación de : {best_solution.valid_metric}")

In [ ]:
print(f"El código de la mejor solución es: \n------------------------------------------------------------\n\n {best_solution.code}")

Probamos a ejecutar el código que nos ha proporcionado para ver los resultados:

In [2]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available. Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is not available.


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim

# Define transformations
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Load the dataset
dataset = ImageFolder("./Data/PlantVillage/", transform=transform)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet50 model
model = resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Adapting to 3 classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / train_size
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# Evaluating the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the validation set: {accuracy:.2f}%")


Epoch 1, Loss: 0.0243
Epoch 2, Loss: 0.0000
Epoch 3, Loss: 0.0000
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 0.0000
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000
Accuracy on the validation set: 100.00%


Probablemente debearíamos haber puesto menos iteraciones para que el entrenamiento no durase poco más de una hora pero así es como ha sido. 

AIDE ha sido capaz de lograr una precisión perfecta para predecir el dataset de patatas con enfermedad.